In [26]:
!pip3 install oandapyV20

  Created wheel for oandapyV20: filename=oandapyV20-0.6.3-py3-none-any.whl size=69747 sha256=1044af92eb6d9d7b54abc688fcafcc7eedef5468650acc20dcc46feb0537ed59
  Stored in directory: c:\users\ritesh kumar singh\appdata\local\pip\cache\wheels\4c\14\a4\a5c83c7d468798f9d4b0032fed8c4cdc9d88e021334a4eccba
Successfully built oandapyV20


In [324]:
class Pakages_required:
    import time
    import requests
    import json
    from oandapyV20 import API
    import oandapyV20.endpoints.orders as orders
    from oandapyV20.contrib.requests import MarketOrderRequest
    import threading
    import logging
    

class Access_cred(Pakages_required):
    def __init__(self):
        accountid = "101-011-17653008-001"
        token = "bc38d46445d60b06ef7dd0a1307983a0-d82e8844d36cb718226d5bcf41a21dba"
        site = "api-fxpractice.oanda.com"
        header = "Bearer "+token
        
        
    def generate_signal(self,ins):
        main.lgger.info("generate_signal function triggered")
        query = {"instruments":ins, "granularity":"S5", "dailyAlignment":"19"}
        path = f"/v3/instruments/{query['instruments']}/candles"
        res_cp = requests.get("https://"+site+path, headers=header, params=query)
        data = res_cp.json()
        cur_price = data['candles'][-1]['mid']['o']
        a = time.localtime()
        b = time.mktime((a.tm_year, a.tm_mon, a.tm_mday, 0, 0, 0, a.tm_wday, a.tm_yday, a.tm_isdst))
        f = time.mktime(a)-b
        query1 = {"instruments":ins, "granularity":"M15", "from":f}
        res_orb = requests.get("https://"+site+path, headers=header, params=query1)
        data = res_orb.json()
        high = data['candles'][-1]['mid']['h']
        low = data['candles'][-1]['mid']['l']
        global g
        if cur_price > high:
            g = 1
        elif cur_price < low:
            g = -1
        else:
            g = 0
            main.lgger.info("no order placed because open price is in the opening range")
        
    def place_order(self,ins,i):
        main.lgger.info("place_order function triggered")
        if i==1:
            client = API(access_token = token)
            mo = MarketOrderRequest(instrument = ins, units = 100)
            json.dumps(mo.data, indent=4)
            {
                "order":{
                    "type":"MARKET",
                    "positionFill":"DEFAULT",
                    "instrument":"EUR_USD",
                    "timeInForce":"FOK",
                    "units":100
                }
            }

            r = orders.OrderCreate(accountid, data= mo.data)
            rv = client.request(r)
            main.lgger.info("buy order placed successfully")
        elif i==-1:
            client = API(access_token = token)
            mo = MarketOrderRequest(instrument = ins, units = -100)
            json.dumps(mo.data, indent=4)
            {
                "order":{
                    "type":"MARKET",
                    "positionFill":"DEFAULT",
                    "instrument":"EUR_USD",
                    "timeInForce":"FOK",
                    "units":-100
                }
            }

            r = orders.OrderCreate(accountid, data= mo.data)
            rv = client.request(r)
            main.lgger.info("sell order placed successfully")
        else:
            pass
        
        
    def last_traded_price(self,ins):
        query = {"instruments":ins, "granularity":"D", "dailyAlignment":"19"}
        path = f"/v3/instruments/{query['instruments']}/candles"
        res_ltp = requests.get("https://"+site+path, headers = header, params = query)
        data = res_ltp.json()
        dump_file = open('dump_file.json', 'a')
        json.dump("Last Traded Price for "+ins+" on "+time.asctime()+" is "+data['candles'][-2]['mid']['c'], dump_file, indent=4)
        dump_file.write('\n')
        dump_file.close()
        main.lgger.info("last traded price dumped")

class main:
    Pakages_required.logging.basicConfig(filename= "log_file.log", format = '%(asctime)s %(message)s', filemode = "w")
    lgger = Pakages_required.logging.getLogger()
    lgger.setLevel(Pakages_required.logging.INFO)
    ins = ["EUR_USD","AUD_USD","NZD_USD"]
    oj = Access_cred()
    for i in ins:
        t1 = threading.Thread(target=oj.generate_signal, args=(i,))
        t2 = threading.Thread(target=oj.last_traded_price, args=(i,))
        t3 = threading.Thread(target=oj.place_order, args=(i,g,))
        t1.start()
        lgger.info("thread 1 started")
        t2.start()
        lgger.info("thread 2 started")
        t3.start()
        lgger.info("thread 3 started")